In [27]:
import sort_RAFT_table as srt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit
import itertools

In [28]:
# ToDo: Kinetik Kurven erstellen,
#  90% oder max als max gemessen als conversion, als Beschreibung für diese nehmen,
#  in ist Monomer und Mn dann
#       sortieren nach Umsatz dahingehend dann nach kürzester Zeit
#       Tabellenausgabe dazu Kinetik Kurve, reaktionsbedingungen (LM, Raft agent, Zeit...)
#  am besten als interaktive website Tabelle.

# get all conversion values,
conversion_list = []
for column in srt.df.columns:
    if "conversion" in column:
        conversion_list.append(column)
print(conversion_list)

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))

print(hours_list)

['t0h-conversion', 't1h-conversion', 't2h-conversion', 't4h-conversion', 't8h-conversion', 't15h-conversion']
[0, 1, 2, 4, 8, 15]


In [29]:
def change_timeformat(time_format):
    h_m_s = str(time_format).split(":")
    m_format = int(h_m_s[0]) * 60 + int(h_m_s[1]) + int(h_m_s[2]) / 60
    return m_format

def change_timeformat_h(time_format):
    h_m_s = str(time_format).split(":")
    h_format = int(h_m_s[0]) + (int(h_m_s[1]) + int(h_m_s[2]) / 60) /60
    return h_format

# creating a table as a lookup to correct all sample measurement times
exact_times = pd.read_excel(srt.INPUT_FILE_PATH, sheet_name="exact sampling times")

time_correction_df = pd.DataFrame(data=exact_times.iloc[3:12, 3:])
time_correction_df.columns = exact_times.loc[2][3:]

time_correction_df.reset_index(inplace=True, drop=True)
ext_time_corr_df_data =[]
for row in time_correction_df.iterrows():
    reactor_nr = row[1]["Reactor"]
    if type(reactor_nr) == int or reactor_nr == "closing of reactors":
        row[1]["Reactor"] = str(reactor_nr)
        ext_time_corr_df_data.append(row[1])
    else:
        reactor_nr = (row[1]["Reactor"]).split("+")
        for nr in reactor_nr:
            row[1]["Reactor"] = nr
            ext_time_corr_df_data.append(row[1].copy())
ext_time_corr_df = pd.DataFrame(data=ext_time_corr_df_data, columns=exact_times.loc[2][3:])
ext_time_corr_df.reset_index(drop=True, inplace=True)
print("The timer was reset to 0 after the first closing of reactors in the \"t = 0\"-sampling-step")
ext_time_corr_df.columns = ["Reactor", 0, 1, 2, 4, 6, 8, 10, 15]

# change time format to minutes and set
time_cols = ext_time_corr_df.columns.difference(["Reactor"])
ext_time_corr_df[time_cols] = ext_time_corr_df[time_cols].apply(lambda x: [change_timeformat_h(d) for d in x])
ext_time_corr_df[0] = ext_time_corr_df[0].apply(lambda x: 0 )

# set index to reactor
ext_time_corr_df.set_index("Reactor", inplace=True)

# Sampling 6h and 10h were only used for sec not NMR -> no conversion for these.
skip_hours_mask = [True if h not in hours_list else False for h in ext_time_corr_df.columns]
ext_time_corr_df.drop(columns=time_cols[skip_hours_mask], inplace=True)

ext_time_corr_df



The timer was reset to 0 after the first closing of reactors in the "t = 0"-sampling-step


,0,1,2,4,8,15
Reactor,,,,,,
1,0,1.005556,2.011111,4.018056,8.016667,15.016667
2,0,1.005556,2.011111,4.018056,8.016667,15.016667
3,0,1.073611,2.068056,4.077778,8.090278,15.098611
4,0,1.073611,2.068056,4.077778,8.090278,15.098611
5,0,1.131944,2.125000,4.143056,8.180556,15.200000
6,0,1.131944,2.125000,4.143056,8.180556,15.200000
7,0,1.191667,2.184722,4.205556,8.256944,15.291667
8,0,1.191667,2.184722,4.205556,8.256944,15.291667
9,0,1.251389,2.241667,4.273611,8.338889,15.381944


In [30]:
# plotting the kinetic curves of the RAFT polymerization
# generate a reformatted table with a entries per sample analysis taken described with a column of the right time

# create a dataframe with just the information needed to append to
kinetic_curves = []
for index, polymerisation_kinetic in srt.df.iterrows():
    # get the right hours list
    # polymerisation_kinetic["reactor"]

    kinetic_curve_entries = pd.DataFrame(
        {"conversion" : polymerisation_kinetic[conversion_list], "time" : hours_list, "reactor" : polymerisation_kinetic["reactor"]})
    kinetic_curve_entries["experiment"] = str(index)+"_"+polymerisation_kinetic["monomer"]+"_"+polymerisation_kinetic["RAFT-Agent"]+"_"+polymerisation_kinetic["solvent"]

    # the times are dependent on the current reactor, get current
    current_reactor_nr = str(polymerisation_kinetic["reactor"])
    current_time_list = ext_time_corr_df.loc[current_reactor_nr]
    kinetic_curve_entries["time"] = list(current_time_list)

    kinetic_curves.append(kinetic_curve_entries)

kinetic_curves_df = pd.concat(kinetic_curves)

In [31]:
len(kinetic_curves) # was 325 before getting away the average below 0.1, now its 317
# kinetic_curves[1]["experiment"].iloc[1]
# px.line(kinetic_curves_df[:20], x="time", y="conversion", color="experiment")

317

In [32]:
# and fit curve and the derivation
# find the maximum of the derivation

'''
deprecated: more theoretical accurate version but practically worse applicable
    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)


def sigmoid_derivative(x, L ,x0, k, b): # ToDo: check if this is the correct derivative
    y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
    return (y)
'''
def neg_growth(x, L, k):
    y = L * (1 - np.exp(k*(-x)))
    return y

def neg_growth_derivative(x, L, k):
    y = L * k * np.exp(k*(-x))
    return y

def linear_growth(x, m):
    y = m * x
    return y

def linear_growth_derivative(x, m):
    return m

xrange = np.arange(-0.2, 5, 0.1)
example_fig = px.line()
for testparams in ([1,1], [2,1], [1,2]):
    example_fig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams))
    example_fig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5, line=dict(dash="dot"))

print("visualisation of the functions")
# example_fig.show()

visualisation of the functions


In [34]:
fig = px.line(title="Kinetic Curve Fit", labels={"x":"time", "y":"conversion"})
hard_to_fit = [] # array for kinetic fits that meet a filter criteria further down the code
colors = px.colors.qualitative.Plotly # set up a simple color palette
extended_xdata = np.linspace(-1, 16.5, 100) # x data array for

# create new dataframe with kinetics per row
# kinetic_curve_columns = ["Experiment", "max_conversion", "reaction_end", "squared_error"]
kinetics_df = pd.DataFrame()

def add_fits_to_plot(fit_func, fit_func_derivative, fit_func_params, marker_dic):
    fig.add_scatter(
        x=extended_xdata, y=fit_func(extended_xdata, *fit_func_params),
        opacity=1, line=dict(dash="dot"), name=f"{fit_func.__name__} fit", marker=marker_dic)
    fig.add_scatter(
        x=extended_xdata, y=fit_func_derivative(extended_xdata, *fit_func_params),
        opacity=0.3, line=dict(dash="dash"), name=f"{fit_func_derivative.__name__}", marker=marker_dic)

def fit_and_exclude_outliers(x, y, fit_func, p0, bounds, nan_policy="omit", iteration=1, outliers=None):
    outliers = outliers if outliers is not None else []

    cf_data = curve_fit(f=fit_func, xdata=x, ydata=y, p0=p0, nan_policy=nan_policy, maxfev=800 * 10, bounds=bounds)

    # calculate the fit points
    fit_points = np.array([fit_func(x, *cf_data[0]) for x in x])
    # calculate the standard deviation of the residuals between the fit and the datapoints
    sigma = np.std(fit_points - y)

    # exclude the outliers
    mask = ~(np.abs(fit_points - y) > 2 * sigma)
    if not mask.all():
        return fit_and_exclude_outliers(x=x[mask], y=y[mask], fit_func=fit_func, p0=cf_data[0], bounds=bounds, nan_policy=nan_policy, iteration=iteration+1, outliers=outliers + [x[~mask]])

    result = {"x": x, "y": y, "p_opt" : cf_data[0], "p_cov" : cf_data[1], "excluded_points" : (x[~mask],y[~mask]), "iteration" : iteration, "outliers" : outliers}
    return result

for idx, kinetic_curve in enumerate(kinetic_curves):
    # first make sure the datapoints are in the right format and not sometimes int sometimes float
    ydata = np.array(kinetic_curve["conversion"].values, dtype=float)
    xdata = np.array(kinetic_curve["time"].values, dtype=float)
    # exclude the nan values from the data
    mask = ~np.isnan(ydata)
    xdata, ydata = xdata[mask], ydata[mask]


    # choosing a few test function
    # if not idx in (316, 301, 302):
    #     continue

    # fitting section
    p0 = [max(ydata), 0.1] # this is a mandatory initial guess
    ng_fit = fit_and_exclude_outliers(x=xdata, y=ydata, fit_func=neg_growth, p0=p0, bounds=([0, -np.inf], [1, np.inf]))
        # f=neg_growth, xdata=xdata, ydata=ydata, p0=p0, nan_policy="omit", maxfev=800*10, bounds=([0, -np.inf], [1, np.inf]))
    popt, pcov = ng_fit["p_opt"], ng_fit["p_cov"]
    ydata, xdata = ng_fit["y"], ng_fit["x"]

    # if ng_fit["iteration"] > 1:
    #     print(idx, ng_fit["p_cov"])

        # fit with linear growth
    p0_l = [max(ydata)/7]
    l_fit = fit_and_exclude_outliers(x=xdata, y=ydata, fit_func=linear_growth, p0=p0_l, bounds=([0], [np.inf]))
    popt_l, pcov_l = l_fit["p_opt"], l_fit["p_cov"]

    # calculate the squared error of fit and data points
    y_st =[]
    x_st = []
    for y_val, x_val in zip(ydata, xdata):
        if np.isnan(y_val) or np.isnan(x_val):
            continue
        y_st.append(y_val)
        x_st.append(x_val)
    y_st = np.array(y_st)
    x_st = np.array(x_st)
    squared_error = np.sum((y_st - neg_growth(x_st, *popt))**2)
    # print("{:.6f}".format(squared_error), str(idx) + ": The optimal parameters are " + " and ".join("%.3f" % x for x in popt) + ". The variance is " + " and respectively ".join("{:.3f} {:.3f}".format(x, y) for x, y in pcov) + ".")

    # ToDo: fill new dataframe
    new_row = pd.DataFrame({"Experiment":kinetic_curve["experiment"].iloc[1], "max_conversion":max(ydata),
                        "reaction_end":"yet to calc", "fit_p1":[popt[0]],"fit_p2":[popt[1]],
                            "p1_err1":[pcov[0][0]],"p1_err2":[pcov[0][1]],"p2_err1":[pcov[1][0]],"p2_err2":[pcov[1][1]], "squared_error":squared_error, "xdata":[xdata], "ydata":[ydata]})
    kinetics_df = pd.concat([kinetics_df, new_row])

    # filter for inspection
    # if not ((popt[1]) < 0.001).any(): continue
    # hard_to_fit.append(idx)

''' comment out if no plot is needed.
    marker_dict = dict(color=colors[idx%len(colors)]) # set same colors per kinetic
    fig.add_scatter(x=xdata, y=ydata, mode="lines+markers", opacity=1, name=kinetic_curve["experiment"].iloc[1], marker=marker_dict)
    add_fits_to_plot(neg_growth, neg_growth_derivative, popt, marker_dict)
    # add_fits_to_plot(linear_growth, linear_growth_derivative, [popt_l], marker_dict)


fig.update_layout(
    yaxis=dict(
        range=[-0.1,1]
    ),
    xaxis_title="Time [h]",
    yaxis_title="Conversion [%]"
)
fig.show()
# '''

print(hard_to_fit, len(hard_to_fit)/len(kinetic_curves))
kinetics_df.reset_index(drop=True, inplace=True)
kinetics_df.drop(axis="index", index=kinetics_df[kinetics_df["max_conversion"] <= 0].index, inplace=True)
kinetics_df.reset_index(drop=True, inplace=True)
kinetics_df

C:\Users\fo72qev\AppData\Local\mambaforge\envs\PFP2\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning:

Covariance of the parameters could not be estimated



[] 0.0


,Experiment,max_conversion,reaction_end,fit_p1,fit_p2,p1_err1,p1_err2,p2_err1,p2_err2,squared_error,xdata,ydata
0,0_6_A_DMF,0.516574,yet to calc,0.776540,0.072249,0.002818,-0.000396,-0.000396,0.000057,1.948374e-04,"[0.0, 2.011111111111111, 4.018055555555556, 8....","[0.0, 0.10436494913029215, 0.2057761732851986,..."
1,1_6_A_DMSO,0.697321,yet to calc,0.814980,0.128174,0.000055,-0.000016,-0.000016,0.000005,2.663440e-05,"[0.0, 2.011111111111111, 4.018055555555556, 8....","[0.0, 0.18921601058551107, 0.327820046311611, ..."
2,2_6_A_Tol,0.323281,yet to calc,1.000000,0.025778,0.004197,-0.000128,-0.000128,0.000004,3.354045e-06,"[0.0, 4.0777777777777775, 8.090277777777779, 1...","[0.0, 0.09824902723735407, 0.1877431906614786,..."
3,3_7_A_DMF,0.565125,yet to calc,0.953088,0.059934,0.009147,-0.000816,-0.000816,0.000074,4.525607e-04,"[0.0, 1.073611111111111, 2.0680555555555555, 4...","[0.0, 0.06908094948265364, 0.09403530127814974..."
4,4_7_A_DMSO,0.790565,yet to calc,1.000000,0.097675,0.004018,-0.000699,-0.000699,0.000129,6.790071e-04,"[0.0, 1.1319444444444444, 2.125, 8.18055555555...","[0.0, 0.11448317307692302, 0.18088942307692302..."
...,...,...,...,...,...,...,...,...,...,...,...,...
309,312_9_E_DMF,0.734328,yet to calc,0.796486,0.165271,0.000009,-0.000004,-0.000004,0.000002,1.305254e-05,"[0.0, 1.2513888888888889, 2.2416666666666667, ...","[0.0, 0.14925373134328357, 0.24477611940298516..."
310,313_9_E_DMSO,0.546547,yet to calc,0.929680,0.204371,0.000025,-0.000008,-0.000008,0.000002,4.779775e-07,"[0.0, 1.3125, 2.301388888888889, 4.33611111111...","[0.0, 0.21921921921921916, 0.3483483483483484,..."
311,314_9_E_Tol,0.360000,yet to calc,0.383237,0.197675,0.000560,-0.000579,-0.000579,0.000826,1.283833e-03,"[0.0, 1.3125, 2.301388888888889, 4.33611111111...","[0.0, 0.11142857142857143, 0.11714285714285722..."
312,315_10_E_DMF,0.840426,yet to calc,0.928939,0.154520,0.001383,-0.000441,-0.000441,0.000173,1.803572e-03,"[0.0, 1.3736111111111111, 2.3694444444444445, ...","[0.0, 0.14893617021276595, 0.2695035460992907,..."


In [35]:
# normalize the errors by dividing them by their respective standard deviation
def normalize_errors(errors):
    return errors / np.std(errors)

for error in ["squared_error", "p1_err1", "p1_err2", "p2_err1", "p2_err2"]:
    kinetics_df[error] = normalize_errors(kinetics_df[error])

kinetics_df

,Experiment,max_conversion,reaction_end,fit_p1,fit_p2,p1_err1,p1_err2,p2_err1,p2_err2,squared_error,xdata,ydata
0,0_6_A_DMF,0.516574,yet to calc,0.776540,0.072249,1.311623e-06,-1.871387e-07,-1.871387e-07,2.106476e-18,0.021570,"[0.0, 2.011111111111111, 4.018055555555556, 8....","[0.0, 0.10436494913029215, 0.2057761732851986,..."
1,1_6_A_DMSO,0.697321,yet to calc,0.814980,0.128174,2.564319e-08,-7.581180e-09,-7.581180e-09,1.941992e-19,0.002949,"[0.0, 2.011111111111111, 4.018055555555556, 8....","[0.0, 0.18921601058551107, 0.327820046311611, ..."
2,2_6_A_Tol,0.323281,yet to calc,1.000000,0.025778,1.953369e-06,-6.027800e-08,-6.027800e-08,1.423882e-19,0.000371,"[0.0, 4.0777777777777775, 8.090277777777779, 1...","[0.0, 0.09824902723735407, 0.1877431906614786,..."
3,3_7_A_DMF,0.565125,yet to calc,0.953088,0.059934,4.257136e-06,-3.856118e-07,-3.856118e-07,2.726124e-18,0.050102,"[0.0, 1.073611111111111, 2.0680555555555555, 4...","[0.0, 0.06908094948265364, 0.09403530127814974..."
4,4_7_A_DMSO,0.790565,yet to calc,1.000000,0.097675,1.870001e-06,-3.304640e-07,-3.304640e-07,4.740432e-18,0.075172,"[0.0, 1.1319444444444444, 2.125, 8.18055555555...","[0.0, 0.11448317307692302, 0.18088942307692302..."
...,...,...,...,...,...,...,...,...,...,...,...,...
309,312_9_E_DMF,0.734328,yet to calc,0.796486,0.165271,4.073759e-09,-1.669983e-09,-1.669983e-09,6.595966e-20,0.001445,"[0.0, 1.2513888888888889, 2.2416666666666667, ...","[0.0, 0.14925373134328357, 0.24477611940298516..."
310,313_9_E_DMSO,0.546547,yet to calc,0.929680,0.204371,1.150770e-08,-3.661012e-09,-3.661012e-09,9.053404e-20,0.000053,"[0.0, 1.3125, 2.301388888888889, 4.33611111111...","[0.0, 0.21921921921921916, 0.3483483483483484,..."
311,314_9_E_Tol,0.360000,yet to calc,0.383237,0.197675,2.606338e-07,-2.736153e-07,-2.736153e-07,3.025748e-17,0.142131,"[0.0, 1.3125, 2.301388888888889, 4.33611111111...","[0.0, 0.11142857142857143, 0.11714285714285722..."
312,315_10_E_DMF,0.840426,yet to calc,0.928939,0.154520,6.437870e-07,-2.082873e-07,-2.082873e-07,6.333258e-18,0.199671,"[0.0, 1.3736111111111111, 2.3694444444444445, ...","[0.0, 0.14893617021276595, 0.2695035460992907,..."


In [36]:
# violin plots of the errors
kinetics_df_positive_errors = kinetics_df
for errors in ["p1_err1", "p1_err2", "p2_err1", "p2_err2"]:
    kinetics_df_positive_errors[errors] = kinetics_df_positive_errors[errors].apply(lambda x: np.absolute(x))
fig = px.violin(kinetics_df_positive_errors, y=["squared_error", "p1_err1", "p1_err2", "p2_err1", "p2_err2"], box=True, points="all", log_y=True)
fig.update_layout(title="Errors normalize by \u03C3", xaxis_title="Error type", yaxis_title="value", yaxis=dict(title="log(value)", range=(-22, 3)))
fig.show()


In [43]:
# plot entry 301 from the dataset
iks = [30, 298, 231, 203]
fig = px.line(title="Kinetic Curve Fit", labels={"x":"time", "y":"conversion"})
for ik in iks:
    marker_dict = dict(color=colors[ik%len(colors)])
    fit_data = kinetics_df.iloc[ik]
    xdata, ydata = fit_data["xdata"], fit_data["ydata"]
    fig.add_scatter(x=xdata, y=ydata, mode="lines+markers", name=kinetic_curves[ik]["experiment"][0], marker=marker_dict)
    add_fits_to_plot(neg_growth, neg_growth_derivative, [fit_data["fit_p1"], fit_data["fit_p2"]], marker_dict)
    fig.update_layout(yaxis=dict(range=[-0.1,1]), xaxis_title="Time [h]", yaxis_title="Conversion [%]")
fig.show()

C:\Users\fo72qev\AppData\Local\Temp\ipykernel_32\1755825589.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [38]:
# kinetics_df[kinetics_df["squared_error"]>0.01] 29/317 entries
# calculating covariance between errors to use only the reasonable ones

# permutate all combinations of the errors
errorcombs = [[],[],[]]
for err1, err2 in itertools.combinations(["squared_error", "p1_err1", "p1_err2", "p2_err1", "p2_err2"], 2):
    # print(f"The covariance between {err1} and {err2} is {np.cov(kinetics_df[err1], kinetics_df[err2])}")
    # print(f"The correlation between {err1} and {err2} is {np.correlate(kinetics_df[err1], kinetics_df[err2])}")
    # print("\n")
    errorcombs[0].append(f"{err1}/{err2}")
    errorcombs[1].append(np.cov(kinetics_df[err1], kinetics_df[err2]))
    errorcombs[2].append(*np.correlate(kinetics_df[err1], kinetics_df[err2]))
errorcombs_dc = {"name": errorcombs[0], "covariance":errorcombs[1], "correlation":errorcombs[2]}
errorcombs_df = pd.DataFrame(data=errorcombs_dc)
errorcombs_df["i_correlation"] = errorcombs_df["correlation"] * (-1)

# plot barplot per pair of errors with superimposed correlation
err_fig = px.bar(title="Correlation between Errors", labels={"correlation":"correlation"}, log_y=True)
err_fig.add_bar(x=errorcombs_df["name"], y=errorcombs_df["correlation"], name="postive correlation", marker_color="green")
err_fig.add_bar(x=errorcombs_df["name"], y=errorcombs_df["i_correlation"], name="negative correlation", marker_color="crimson")
err_fig.show()

In [39]:
# descry when a function aligns to the datapoints in a reasonable way
#   Hence, wheneth' the blunder exaggerates, an 80% betweeneth' of the maximum conversion in that kinetic should be assessed to be the maximum conversion.
